In [1]:
%load_ext autoreload
%autoreload 2

In [37]:
import numpy as np
import os.path as path

import matplotlib.pyplot as plt
%matplotlib inline
import sklearn.preprocessing as pre
import librosa
import music21 as mus

import vmo
import vmo.analysis as van
import vmo.generate as vge
# Music analysis
import vmo.utils.chromagram as vchroma
import vmo.utils.music21_interface as vmusic
import vmo.utils.harmonic_changes as vharm
# Logics
import vmo.logics.model_checking as vmodel

In [19]:
midi_path = path.expanduser('~/Musique/midi/')
midi_file = 'mozart/sonatina_num_4-mvt_1.mid'

"""Oracle generation."""

# Generate music21 stream from MIDI file
stream = mus.converter.parse(midi_path + midi_file)
# Generate VMO from music21 stream
oracle = vmusic.from_stream(stream, framesize=1.0, dfunc='tonnetz')

In [44]:
"""Harmonic changes detection and chord progression extraction."""

# Extract the sequences of notes from the stream
notes = stream.flat.notes
# Extract the first 16 quarter-notes
_beginning_notes = vmusic.extract_frame(notes, 0., 16.)
# Compute offsets of harmonic changes (in quarter-length)
harmonic_changes = vharm.from_stream_by_offsets(
    _beginning_notes, framesize=1/4.)
# Compute the chord progression in the extracted frame
prog = vmusic.get_chord_progression(_beginning_notes)

print("First two measures:")
_beginning_notes.show('text')
print("\nExtracted progression:\n" + str(prog))
print("\nHarmonic changes:\n" + str(harmonic_changes))

First two measures:
{0.0} <music21.chord.Chord D5 F5>
{1.0} <music21.chord.Chord D5 F5>
{2.0} <music21.chord.Chord D5 F5>
{2.0} <music21.note.Note B->
{4.0} <music21.chord.Chord E-5 G5>
{4.5} <music21.chord.Chord F5 A5>
{5.0} <music21.chord.Chord G5 B-5>
{5.5} <music21.chord.Chord E-5 G5>
{6.0} <music21.chord.Chord D5 F5>
{6.0} <music21.note.Note B->
{8.0} <music21.note.Note B->
{8.0} <music21.chord.Chord D4 F4>
{9.0} <music21.note.Note B->
{9.0} <music21.chord.Chord D4 F4>
{10.0} <music21.note.Note C>
{10.0} <music21.note.Note E->
{10.0} <music21.note.Note F>
{10.75} <music21.note.Note A>
{11.0} <music21.note.Note B->
{11.0} <music21.note.Note D>
{12.0} <music21.note.Note F>
{12.0} <music21.note.Note A>
{12.5} <music21.note.Note E>
{13.0} <music21.chord.Chord C5 E-5>
{13.0} <music21.note.Note B->
{13.5} <music21.chord.Chord B-4 D5>
{14.0} <music21.chord.Chord C5 E-5>
{14.0} <music21.note.Note F>
{14.5} <music21.chord.Chord B-4 D5>
{15.0} <music21.chord.Chord A4 C5>

Extracted progress

In [45]:
"""Integrated MIDI playback.

Might need some parameters-tweaking to work if midi player binding is not
done straight out-of-the-box with music21.

In that case, run:
>>> from music21 import *
>>> us = environment.UserSettings()
>>> us.create()
>>> us['midiPath'] = <PATH/TO/MIDI/PLAYER>
>>> # Example : us['midiPath'] = u'/usr/bin/totem-audio-preview'
"""

_beginning_notes.show('midi')

In [46]:
"""Chord-progression extraction."""

# Two sequences of degrees, we'll try to generate a path following the first
# then the second (possibibly with an arbitrary path between both) in `oracle`
simple_progs = [[1, 3, 5, 3, 5], [1, 5, 3, 5, 3]]
path, chosen_tonic = vmodel.make_piecewise_chord_progression_tonic_free(
    oracle, simple_progs)
for state in path:
    print(state)

# Recompose new stream from extracted path
extracted = vmusic.path_to_stream(notes, path)

TypeError: unsupported operand type(s) for -: 'dict' and 'int'

{'s': '0', 'pitchRoot': 'p_Init'}
{'s': '191', 'pitchRoot': 'p_E-'}
{'s': '116', 'pitchRoot': 'p_G'}
{'s': '127', 'pitchRoot': 'p_B-'}
{'s': '116', 'pitchRoot': 'p_G'}
{'s': '127', 'pitchRoot': 'p_B-'}
{'s': '191', 'pitchRoot': 'p_E-'}
{'s': '127', 'pitchRoot': 'p_B-'}
{'s': '116', 'pitchRoot': 'p_G'}
{'s': '127', 'pitchRoot': 'p_B-'}
{'s': '116', 'pitchRoot': 'p_G'}
